# Exercise - Autoencoder

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. We will build an autoencoder to learn the representation of good loans. That way, we can see if an unknown loan will be a good or bad loan based on its reconstruction error.  

Note: in the data set, we don't have a column that indicates whether a loan is "good" or "bad". So, we can't train a classification model like we did before.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
good = pd.read_csv('good loans.csv')

unknown = pd.read_csv('unknown loans.csv')

In [3]:
# Note that there is no GOOD/BAD classification. Though, we know that
# these loans are good

good.head()

,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076
3,25300,62540.0,101165.0,DebtCon,ProfExe,0.0,0.0,0.0,195.451331,0.0,25.0,35.200865
4,27700,73148.0,101462.0,DebtCon,ProfExe,10.0,0.0,0.0,264.605389,0.0,33.0,40.475793


In [4]:
good.shape

(1489, 12)

In [5]:
unknown.shape

(1011, 12)

# Data Prep

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

##  Identify the numeric, binary, and categorical columns

In [7]:
# Identify the numerical columns
numeric_columns = good.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = good.select_dtypes('object').columns.to_list()

In [8]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [9]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [10]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [11]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [12]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],   
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for GOOD

In [13]:
#Fit and transform the train data
good_x = preprocessor.fit_transform(good)

good_x

array([[ 0.6365782 , -0.29793911, -0.12177109, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6546228 ,  1.02893596,  1.62034212, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.18546313, -0.25851565, -0.25756212, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 4.18234262, -0.04383447,  0.81444926, ...,  0.        ,
         0.        ,  0.        ],
       [-1.30321659, -0.47828045, -0.73681873, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03208401, -0.46521555, -0.38146895, ...,  0.        ,
         0.        ,  0.        ]])

In [14]:
good_x.shape

(1489, 20)

# Tranform: transform() for UNKNOWN

In [15]:
# Transform the test data
unknown_x = preprocessor.transform(unknown)

unknown_x

array([[ 2.81095281,  3.755331  ,  3.98956663, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.32079765, -0.54180025, -0.65167516, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19448543,  0.59006724,  0.48213012, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.79796772,  0.05628738,  0.05541527, ...,  0.        ,
         0.        ,  0.        ],
       [-0.16640662, -0.510536  , -0.45504735, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81601232, -0.92230904, -0.93360606, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
unknown_x.shape

(1011, 20)

# Build an Autoencoder to learn the representation of GOOD loans

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
model = keras.models.Sequential()

#Encoder
model.add(keras.layers.InputLayer(input_shape=20))
model.add(keras.layers.Dense(14, activation='relu'))
model.add(keras.layers.Dense(11, activation='relu'))

#Decoder
model.add(keras.layers.Dense(14,activation='relu'))
model.add(keras.layers.Dense(20,activation=None))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                294       
                                                                 
 dense_1 (Dense)             (None, 11)                165       
                                                                 
 dense_2 (Dense)             (None, 14)                168       
                                                                 
 dense_3 (Dense)             (None, 20)                300       
                                                                 
Total params: 927
Trainable params: 927
Non-trainable params: 0
_________________________________________________________________


In [19]:
adam = keras.optimizers.Adam(learning_rate=0.001)


model.compile(loss='mse', optimizer='Nadam', metrics=['mean_squared_error'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [21]:
# Be careful: both input and output are "housing_normal_std" while training the autoencoder

model.fit(good_x, good_x, 
          validation_data = (good_x, good_x),
          epochs=100, batch_size=20, callbacks=callback)

Epoch 1/100
75/75 [==============================] - 4s 10ms/step - loss: 0.6017 - mean_squared_error: 0.6017 - val_loss: 0.5460 - val_mean_squared_error: 0.5460
Epoch 2/100
75/75 [==============================] - 0s 6ms/step - loss: 0.5108 - mean_squared_error: 0.5108 - val_loss: 0.4701 - val_mean_squared_error: 0.4701
Epoch 3/100
75/75 [==============================] - 0s 6ms/step - loss: 0.4353 - mean_squared_error: 0.4353 - val_loss: 0.3996 - val_mean_squared_error: 0.3996
Epoch 4/100
75/75 [==============================] - 0s 6ms/step - loss: 0.3750 - mean_squared_error: 0.3750 - val_loss: 0.3478 - val_mean_squared_error: 0.3478
Epoch 5/100
75/75 [==============================] - 0s 6ms/step - loss: 0.3249 - mean_squared_error: 0.3249 - val_loss: 0.2979 - val_mean_squared_error: 0.2979
Epoch 6/100
75/75 [==============================] - 0s 5ms/step - loss: 0.2783 - mean_squared_error: 0.2783 - val_loss: 0.2565 - val_mean_squared_error: 0.2565
Epoch 7/100
75/75 [==============

75/75 [==============================] - 0s 6ms/step - loss: 0.0431 - mean_squared_error: 0.0431 - val_loss: 0.0424 - val_mean_squared_error: 0.0424
Epoch 52/100
75/75 [==============================] - 0s 6ms/step - loss: 0.0429 - mean_squared_error: 0.0429 - val_loss: 0.0423 - val_mean_squared_error: 0.0423
Epoch 53/100
75/75 [==============================] - 0s 6ms/step - loss: 0.0422 - mean_squared_error: 0.0422 - val_loss: 0.0423 - val_mean_squared_error: 0.0423
Epoch 54/100
75/75 [==============================] - 0s 6ms/step - loss: 0.0421 - mean_squared_error: 0.0421 - val_loss: 0.0415 - val_mean_squared_error: 0.0415
Epoch 55/100
75/75 [==============================] - 0s 6ms/step - loss: 0.0418 - mean_squared_error: 0.0418 - val_loss: 0.0420 - val_mean_squared_error: 0.0420
Epoch 56/100
75/75 [==============================] - 0s 6ms/step - loss: 0.0415 - mean_squared_error: 0.0415 - val_loss: 0.0407 - val_mean_squared_error: 0.0407
Epoch 57/100
75/75 [=====================

### Check the average MSE on the "good" loans

In [22]:
model.evaluate(good_x, good_x)

47/47 [==============================] - 0s 3ms/step - loss: 0.0335 - mean_squared_error: 0.0335


[0.033524978905916214, 0.033524978905916214]

In [23]:
#Multiply by 100 to make sense of the error term:

model.evaluate(good_x, good_x)[0]*100

47/47 [==============================] - 0s 3ms/step - loss: 0.0335 - mean_squared_error: 0.0335


3.3524978905916214

### Check the average MSE on the "unknown" data

In [24]:
model.evaluate(unknown_x, unknown_x)

32/32 [==============================] - 0s 3ms/step - loss: 0.1048 - mean_squared_error: 0.1048


[0.10479287058115005, 0.10479287058115005]

In [25]:
#Multiply by 100 to make sense of the error term:

model.evaluate(unknown_x, unknown_x)[0]*100

32/32 [==============================] - 0s 3ms/step - loss: 0.1048 - mean_squared_error: 0.1048


10.479287058115005

### Do you think the "unknown" loans look like good loans or not (justify your answer using the interpretation of the average MSE values)

## Predict first 20 in Good loan Data 

In [26]:
#Good Loan
from sklearn.metrics import mean_squared_error

for i in range(0,20):
    prediction = model.predict(good_x[i:i+1])
    print((mean_squared_error(good_x[i:i+1], prediction))*100)

    
#Error terms are multiplied by 100 to make sense of the numbers

1/1 [==============================] - 0s 310ms/step
1.1447358619132628
1/1 [==============================] - 0s 72ms/step
6.710941329281363
1/1 [==============================] - 0s 75ms/step
1.7493886154265483
1/1 [==============================] - 0s 46ms/step
2.794272889846644
1/1 [==============================] - 0s 45ms/step
2.165758455153841
1/1 [==============================] - 0s 54ms/step
1.4540517686304983
1/1 [==============================] - 0s 54ms/step
3.5854488441430563
1/1 [==============================] - 0s 36ms/step
3.8134174342674654
1/1 [==============================] - 0s 46ms/step
4.229177332165989
1/1 [==============================] - 0s 55ms/step
2.214754133679226
1/1 [==============================] - 0s 55ms/step
3.1546653367573967
1/1 [==============================] - 0s 47ms/step
1.8526556911548289
1/1 [==============================] - 0s 43ms/step
0.7799799842358753
1/1 [==============================] - 0s 61ms/step
2.158703231368238
1/1 [======

## Predict first 20 in Unknown Loan Data

In [27]:
#Unknown Loan
for i in range(0,20):
    prediction = model.predict(unknown_x[i:i+1])
    print((mean_squared_error(unknown_x[i:i+1], prediction))*100)

    
#Error terms are multiplied by 100 to make sense of the numbers

1/1 [==============================] - 0s 60ms/step
8.710497619745567
1/1 [==============================] - 0s 61ms/step
5.149779130082891
1/1 [==============================] - 0s 42ms/step
9.108267697424113
1/1 [==============================] - 0s 45ms/step
1.6268869992382868
1/1 [==============================] - 0s 41ms/step
6.9620073390165444
1/1 [==============================] - 0s 46ms/step
4.1956270173706
1/1 [==============================] - 0s 53ms/step
3.1749272924612617
1/1 [==============================] - 0s 48ms/step
1.8754560078300029
1/1 [==============================] - 0s 58ms/step
179.36922748268097
1/1 [==============================] - 0s 50ms/step
8.831552561292877
1/1 [==============================] - 0s 52ms/step
4.4913470610409
1/1 [==============================] - 0s 53ms/step
4.035887130260505
1/1 [==============================] - 0s 60ms/step
6.757630305654952
1/1 [==============================] - 0s 41ms/step
1.0720638454585507
1/1 [=============